In [31]:
import pandas as pd
import numpy as np
import pylab as pl
import os 
import sys
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
%matplotlib inline


In [3]:
#!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-01.csv

--2017-11-20 16:43:06--  https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-01.csv
Resolving s3.amazonaws.com... 52.216.128.253
Connecting to s3.amazonaws.com|52.216.128.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1985964692 (1.8G) [application/octet-stream]
Saving to: “yellow_tripdata_2015-01.csv”

100%[====================================>] 1,985,964,692 42.5M/s   in 43s     

2017-11-20 16:43:50 (43.7 MB/s) - “yellow_tripdata_2015-01.csv” saved [1985964692/1985964692]



In [45]:
#!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2015-01.csv

--2017-11-20 18:51:26--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2015-01.csv
Resolving s3.amazonaws.com... 52.216.64.243
Connecting to s3.amazonaws.com|52.216.64.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243307680 (232M) [application/octet-stream]
Saving to: “green_tripdata_2015-01.csv”

100%[======================================>] 243,307,680 58.3M/s   in 4.2s    

2017-11-20 18:51:30 (54.8 MB/s) - “green_tripdata_2015-01.csv” saved [243307680/243307680]



In [48]:
#!wget https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-01.csv

--2017-11-20 19:06:05--  https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-01.csv
Resolving s3.amazonaws.com... 52.216.97.237
Connecting to s3.amazonaws.com|52.216.97.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85733063 (82M) [text/csv]
Saving to: “fhv_tripdata_2015-01.csv”

100%[======================================>] 85,733,063  46.0M/s   in 1.8s    

2017-11-20 19:06:07 (46.0 MB/s) - “fhv_tripdata_2015-01.csv” saved [85733063/85733063]



In [26]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip

--2017-11-26 18:27:20--  https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip
Resolving s3.amazonaws.com... 52.216.229.133
Connecting to s3.amazonaws.com|52.216.229.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1025147 (1001K) [application/zip]
Saving to: “taxi_zones.zip”

100%[======================================>] 1,025,147   3.38M/s   in 0.3s    

2017-11-26 18:27:21 (3.38 MB/s) - “taxi_zones.zip” saved [1025147/1025147]



In [27]:
!mv taxi_zones.zip data

In [33]:
!unzip data/taxi_zones.zip -d data

Archive:  data/taxi_zones.zip
  inflating: data/taxi_zones.prj     
  inflating: data/taxi_zones.dbf     
  inflating: data/taxi_zones.shp     
  inflating: data/taxi_zones.shp.xml  
  inflating: data/taxi_zones.shx     
  inflating: data/taxi_zones.sbn     
  inflating: data/taxi_zones.sbx     


In [5]:
#!mv yellow_tripdata_2015-01.csv data

In [46]:
#!mv green_tripdata_2015-01.csv data

In [50]:
#!mv fhv_tripdata_2015-01.csv data

In [61]:
taxi_data = pd.read_csv("data/yellow_tripdata_2015-01.csv")

In [3]:
fhv_data = pd.read_csv("data/fhv_tripdata_2015-01.csv")

In [4]:
#taxi_data.head()

NameError: name 'taxi_data' is not defined

In [8]:
#taxi_data.shape

(12748986, 19)

In [11]:
weather_data = pd.read_csv("data/1106040.csv")

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (14,15,20,23,25,40,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
#taxi_data.tpep_pickup_datetime = pd.to_datetime(taxi_data.tpep_pickup_datetime)

In [5]:
def get_times(data):
    x, y = data
    return (x, y)

In [21]:
#taxi_data['day'] = taxi_data.tpep_pickup_datetime.dt.day
#taxi_data['hour'] = taxi_data.tpep_pickup_datetime.dt.hour


In [27]:
#taxi_data['DAY_HOUR'] = taxi_data[['day', 'hour']].apply(get_times, axis=1)

In [28]:
#taxi_data['DAY_HOUR'].head()

0    (15, 19)
1    (10, 20)
2    (10, 20)
3    (10, 20)
4    (10, 20)
Name: DAY_HOUR, dtype: object

In [6]:
fhv_sub = fhv_data[fhv_data.locationID.isnull() == False]

In [7]:
fhv_sub.Pickup_date = pd.to_datetime(fhv_sub.Pickup_date)


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [8]:
fhv_sub['day'] = fhv_sub.Pickup_date.dt.day
fhv_sub['hour'] = fhv_sub.Pickup_date.dt.hour
fhv_sub['DAY_HOUR'] = fhv_sub[['day', 'hour']].apply(get_times, axis=1)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [9]:
fhv_fin = fhv_sub.groupby(['locationID','DAY_HOUR']).count().reset_index()

In [12]:
weather_data.Date = pd.to_datetime(weather_data.DATE)

In [13]:
weather_subdata = weather_data[weather_data.Date.dt.month == 1]

In [14]:
weather_subdata.DATE =  pd.to_datetime(weather_subdata.DATE)

In [15]:
weather_subdata['day'] = weather_subdata.DATE.dt.day
weather_subdata['hour'] = weather_subdata.DATE.dt.hour

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [16]:
weather_subdata['DAY_HOUR'] = weather_subdata[['day','hour']].apply(get_times, axis=1)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
weather_subdata['DAY_HOUR'].head()

8789    (1, 0)
8790    (1, 1)
8791    (1, 2)
8792    (1, 3)
8793    (1, 4)
Name: DAY_HOUR, dtype: object

In [18]:
#merged_taxi = taxi_data.merge(weather_subdata, left_on='DAY_HOUR', right_on='DAY_HOUR')

NameError: name 'taxi_data' is not defined

In [19]:
#merged_taxi.shape

NameError: name 'merged_taxi' is not defined

In [25]:
weather_subdata.columns

Index([u'STATION', u'STATION_NAME', u'ELEVATION', u'LATITUDE', u'LONGITUDE',
       u'DATE', u'REPORTTPYE', u'HOURLYSKYCONDITIONS', u'HOURLYVISIBILITY',
       u'HOURLYPRSENTWEATHERTYPE', u'HOURLYDRYBULBTEMPF',
       u'HOURLYDRYBULBTEMPC', u'HOURLYWETBULBTEMPF', u'HOURLYWETBULBTEMPC',
       u'HOURLYDewPointTempF', u'HOURLYDewPointTempC',
       u'HOURLYRelativeHumidity', u'HOURLYWindSpeed', u'HOURLYWindDirection',
       u'HOURLYWindGustSpeed', u'HOURLYStationPressure',
       u'HOURLYPressureTendency', u'HOURLYPressureChange',
       u'HOURLYSeaLevelPressure', u'HOURLYPrecip', u'HOURLYAltimeterSetting',
       u'DAILYMaximumDryBulbTemp', u'DAILYMinimumDryBulbTemp',
       u'DAILYAverageDryBulbTemp', u'DAILYDeptFromNormalAverageTemp',
       u'DAILYAverageRelativeHumidity', u'DAILYAverageDewPointTemp',
       u'DAILYAverageWetBulbTemp', u'DAILYHeatingDegreeDays',
       u'DAILYCoolingDegreeDays', u'DAILYSunrise', u'DAILYSunset',
       u'DAILYWeather', u'DAILYPrecip', u'DAILYSnowfall

In [43]:
#merged_taxi[['tpep_pickup_datetime','DATE']].head()

,tpep_pickup_datetime,DATE
0,2015-01-15 19:05:39,2015-01-15 19:51:00
1,2015-01-15 19:05:39,2015-01-15 19:51:00
2,2015-01-15 19:05:40,2015-01-15 19:51:00
3,2015-01-15 19:05:40,2015-01-15 19:51:00
4,2015-01-15 19:05:41,2015-01-15 19:51:00


In [20]:
merged_fhv = fhv_fin.merge(weather_subdata, left_on='DAY_HOUR', right_on='DAY_HOUR')

In [21]:
merged_fhv.head()

,locationID,DAY_HOUR,Dispatching_base_num,Pickup_date,day_x,hour_x,STATION,STATION_NAME,ELEVATION,LATITUDE,...,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD,day_y,hour_y
0,1.0,"(1, 15)",1,1,1,1,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
1,2.0,"(1, 15)",1,1,1,1,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
2,4.0,"(1, 15)",19,19,19,19,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
3,5.0,"(1, 15)",1,1,1,1,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
4,7.0,"(1, 15)",16,16,16,16,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15


In [34]:
taxi_zones = gpd.read_file("data/taxi_zones.shp")

In [35]:
taxi_zones.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527103 192536.0856972019,..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"(POLYGON ((1033269.243591294 172126.0078125, 1..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.769506663 256767.6975403726,..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.4667968601 203714.0759887695,..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.3104932606 144283.335850656, ..."


In [36]:
taxi_zones.crs

{u'datum': u'NAD83',
 u'lat_0': 40.16666666666666,
 u'lat_1': 40.66666666666666,
 u'lat_2': 41.03333333333333,
 u'lon_0': -74,
 u'no_defs': True,
 u'proj': u'lcc',
 u'units': u'us-ft',
 u'x_0': 300000,
 u'y_0': 0}

In [37]:
taxi_zones = taxi_zones.to_crs(epsg=2263)

In [39]:
merged_fhv.columns

Index([u'locationID', u'DAY_HOUR', u'Dispatching_base_num', u'Pickup_date',
       u'day_x', u'hour_x', u'STATION', u'STATION_NAME', u'ELEVATION',
       u'LATITUDE', u'LONGITUDE', u'DATE', u'REPORTTPYE',
       u'HOURLYSKYCONDITIONS', u'HOURLYVISIBILITY', u'HOURLYPRSENTWEATHERTYPE',
       u'HOURLYDRYBULBTEMPF', u'HOURLYDRYBULBTEMPC', u'HOURLYWETBULBTEMPF',
       u'HOURLYWETBULBTEMPC', u'HOURLYDewPointTempF', u'HOURLYDewPointTempC',
       u'HOURLYRelativeHumidity', u'HOURLYWindSpeed', u'HOURLYWindDirection',
       u'HOURLYWindGustSpeed', u'HOURLYStationPressure',
       u'HOURLYPressureTendency', u'HOURLYPressureChange',
       u'HOURLYSeaLevelPressure', u'HOURLYPrecip', u'HOURLYAltimeterSetting',
       u'DAILYMaximumDryBulbTemp', u'DAILYMinimumDryBulbTemp',
       u'DAILYAverageDryBulbTemp', u'DAILYDeptFromNormalAverageTemp',
       u'DAILYAverageRelativeHumidity', u'DAILYAverageDewPointTemp',
       u'DAILYAverageWetBulbTemp', u'DAILYHeatingDegreeDays',
       u'DAILYCoolingDegr

In [53]:
merged_fhv.head(20)

,locationID,DAY_HOUR,Dispatching_base_num,Pickup_date,day_x,hour_x,STATION,STATION_NAME,ELEVATION,LATITUDE,...,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD,day_y,hour_y
0,1.0,"(1, 15)",1,1,1,1,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
1,2.0,"(1, 15)",1,1,1,1,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
2,4.0,"(1, 15)",19,19,19,19,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
3,5.0,"(1, 15)",1,1,1,1,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
4,7.0,"(1, 15)",16,16,16,16,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
5,10.0,"(1, 15)",1,1,1,1,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
6,11.0,"(1, 15)",1,1,1,1,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
7,12.0,"(1, 15)",4,4,4,4,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
8,13.0,"(1, 15)",10,10,10,10,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15
9,14.0,"(1, 15)",1,1,1,1,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,...,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,1,15


In [43]:
crs = {'init': 'epsg:4326'}

In [48]:
fhv_sub.locationID = fhv_sub.locationID.astype(int)

In [51]:
fhv_locs = taxi_zones.merge(fhv_sub, left_on='LocationID', right_on='locationID')

In [ ]:
geometry = [Point(xy) for xy in zip(fhv_sub.) ]

In [58]:
fhv_locs.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,Dispatching_base_num,Pickup_date,locationID,day,hour,DAY_HOUR
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527121 192536.0857092953,...",B00053,2015-01-03 19:00:00,1,3,19,"(3, 19)"
1,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527121 192536.0857092953,...",B00053,2015-01-05 10:00:00,1,5,10,"(5, 10)"
2,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527121 192536.0857092953,...",B00053,2015-01-05 18:53:00,1,5,18,"(5, 18)"
3,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527121 192536.0857092953,...",B00053,2015-01-08 21:05:00,1,8,21,"(8, 21)"
4,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527121 192536.0857092953,...",B00053,2015-01-09 14:00:00,1,9,14,"(9, 14)"


In [59]:
fhv_all = fhv_locs.merge(weather_subdata, left_on='DAY_HOUR', right_on='DAY_HOUR')

In [60]:
fhv_all.shape

(3033516, 105)